# Как передаются параметры в функцию

In [1]:
def foo(x)
  x.object_id
end

x = 10
x.object_id == foo(x)

true


true

In [8]:
def foo1(x)
  x += 1
  return x.object_id
end

x = 10
x.object_id == foo1(x)

false

In [9]:
def foo2(x)
  x += [1]
  return x.object_id
end

a = [1, 2, 3]
a.object_id == foo2(a)

false

In [10]:
def foo2(x)
  x << 1
  return x.object_id
end

a = [1, 2, 3]
a.object_id == foo2(a)

true

Подытоживая: все как в начале третьего семинара. Параметры передаются по ссылке, если в функции есть создание нового объекта, то переданный не меняется, если подразумевается изменение существующего (<<), то меняется.

In [13]:
def foo_map(iterable)
  iterable.map {|x| x**2 }
end

def foo_map!(iterable)
  iterable.map! {|x| x**2 }
end

a = [1, 2, 3]
b = [1, 2, 3]

# Создалась копия
p a == foo_map(a)
# Изменился объект
p b == foo_map!(b)

false
true


true